The Goal of this Notebook is to understand the working of Normalizing flows by using the example of some 2D datasets

In [10]:
# imports 
import os
import math
import torch
import numpy as np
import sklearn.datasets
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from tqdm.auto import tqdm 
import contextlib 
import imageio # For creating GIF/Video later

In [11]:
available = torch.cuda.is_available()
device = torch.device("cuda:0" if available else "cpu")
print(f'Using device: {device}')

# --- Data Loading ---
def load_and_preprocess_data(dataset_name, n_samples=2000):
    if dataset_name == 'moons':
        data, _ = sklearn.datasets.make_moons(n_samples=n_samples, noise=0.05)
    elif dataset_name == 'circles':
        data, _ = sklearn.datasets.make_circles(n_samples=n_samples, factor=0.5, noise=0.05)
    else:
        raise ValueError("Unknown dataset name")

    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    return data.astype(np.float32), scaler

Using device: cuda:0


Setting Model Components 

In [12]:
class Dataset2D(Dataset):
    def __init__(self, data):
        self.data = torch.from_numpy(data)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

# --- Model Components ---
# Just using simple MLP for s and t here as the distribution is simple (not complex like images)
def create_mlp(input_dim, output_dim, hidden_units=64, depth=3):
    layers = [nn.Linear(input_dim, hidden_units), nn.ReLU()]
    for _ in range(depth - 1):
        layers.extend([nn.Linear(hidden_units, hidden_units), nn.ReLU()])
    layers.append(nn.Linear(hidden_units, output_dim))
    return nn.Sequential(*layers)

class AffineCouplingLayer(nn.Module):
    def __init__(self, dim, hidden_units=64, depth=3, mask_type='checkerboard', scale_factor=1.0):
        super().__init__(); self.dim = dim; self.mask_type = mask_type
        self.scale_factor = nn.Parameter(torch.tensor(scale_factor))
        if mask_type == 'checkerboard':
            self.dim_keep = dim // 2
            self.dim_transform = dim - self.dim_keep
        elif mask_type == 'channel': 
            self.dim_transform = dim // 2
            self.dim_keep = dim - self.dim_transform
        else: raise ValueError("Unknown mask type")
        self.s_net = create_mlp(self.dim_keep, self.dim_transform, hidden_units, depth)
        self.t_net = create_mlp(self.dim_keep, self.dim_transform, hidden_units, depth)

    def _get_mask(self, x):
        # Returns (dimensions_to_keep, dimensions_to_transform)
        if self.mask_type == 'checkerboard':
             return x[:, :self.dim_keep], x[:, self.dim_keep:]
        elif self.mask_type == 'channel':
             # Keep second half, transform first half (or vice versa)
             return x[:, self.dim_transform:], x[:, :self.dim_transform]

    def _combine(self, x_keep, x_transform):
        # Combines dimensions back in the original order
        if self.mask_type == 'checkerboard':
             return torch.cat((x_keep, x_transform), dim=1)
        elif self.mask_type == 'channel':
             # Combine transformed (first half) and kept (second half) (Real NVP paper eq 2)
             return torch.cat((x_transform, x_keep), dim=1)

    def forward(self, x):
        x_keep, x_transform = self._get_mask(x)
        s = torch.tanh(self.s_net(x_keep)) * self.scale_factor
        t = self.t_net(x_keep)
        y_transform = x_transform * torch.exp(s) + t
        log_det = torch.sum(s, dim=1) # Log determinant of the Jacobian
        y = self._combine(x_keep, y_transform)
        return y, log_det

    def inverse(self, y):
        y_keep, y_transform = self._get_mask(y) # Use same masking logic on y
        s = torch.tanh(self.s_net(y_keep)) * self.scale_factor
        t = self.t_net(y_keep)
        x_transform = (y_transform - t) * torch.exp(-s) # Inverse transformation
        x = self._combine(y_keep, x_transform)
        return x

Defining RealNVP module 

In [13]:
class RealNVP_2D(nn.Module):
    def __init__(self, dim=2, num_coupling_layers=8, hidden_units=64, depth=3):
        super().__init__()
        self.dim = dim
        layers = []
        for i in range(num_coupling_layers):
            mask_type = 'checkerboard' if i % 2 == 0 else 'channel' # Alternate masks (best used in practice)
            layers.append(AffineCouplingLayer(dim, hidden_units, depth, mask_type))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        log_det_total = torch.zeros(x.shape[0], device=x.device)
        y = x
        for layer in self.layers:
            y, log_det = layer(y)
            log_det_total += log_det
        return y, log_det_total 

    def inverse(self, z):
        x = z
        for layer in reversed(self.layers): # Apply layers in reverse for inverse
            x = layer.inverse(x)
        return x

Loss function and Saving Animations

In [14]:
def nll_loss(z, log_det_total):
    log_pz = -0.5 * torch.sum(z**2, dim=1) - 0.5 * z.shape[1] * math.log(2 * math.pi) # Log prob of z under N(0,I)
    log_px = log_pz + log_det_total # Change of variables formula
    nll = -torch.mean(log_px) 
    return nll

# Animation Frame Saving Function 
def save_animation_frame(model, data_points, dataset_name, epoch, device, frame_dir, num_samples=500, grid_res_vis=20):
    model.eval() # Set model to evaluation mode

    fig, axes = plt.subplots(2, 2, figsize=(8, 8))
    plt.suptitle(f'{dataset_name} - Epoch {epoch}', fontsize=14)
    plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0.3, hspace=0.3)

    data_points_np = data_points.cpu().numpy()
    z_lim = 3.5 
    x_lim = (data_points_np[:, 0].min() - 0.5, data_points_np[:, 0].max() + 0.5)
    y_lim = (data_points_np[:, 1].min() - 0.5, data_points_np[:, 1].max() + 0.5)

    # --- Top Left: Original Data (Scaled) ---
    ax = axes[0, 0]
    ax.scatter(data_points_np[:, 0], data_points_np[:, 1], s=5, alpha=0.4, c='blue', label='Input Data')
    ax.set_title('Input Data Space (X)', fontsize=10)
    ax.set_xlabel('x1 (scaled)', fontsize=9); ax.set_ylabel('x2 (scaled)', fontsize=9)
    ax.grid(True, alpha=0.3); ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(x_lim); ax.set_ylim(y_lim) # Use calculated limits
    ax.tick_params(axis='both', which='major', labelsize=8)

    # --- Top Right: Transformed Data & Warped Grid (Data -> Latent) ---
    ax = axes[0, 1]
    with torch.no_grad():
        z, _ = model(data_points)
        z_np = z.cpu().numpy()
        x_range = np.linspace(x_lim[0], x_lim[1], grid_res_vis)
        y_range = np.linspace(y_lim[0], y_lim[1], grid_res_vis)
        grid_x, grid_y = np.meshgrid(x_range, y_range)
        grid_data = torch.from_numpy(np.stack([grid_x.ravel(), grid_y.ravel()], axis=1).astype(np.float32)).to(device)

        grid_z, _ = model(grid_data)
        grid_z_np = grid_z.cpu().numpy()
        # Plot warped grid lines
        for i in range(grid_res_vis):
            ax.plot(grid_z_np[i*grid_res_vis:(i+1)*grid_res_vis, 0], grid_z_np[i*grid_res_vis:(i+1)*grid_res_vis, 1], color='gray', linewidth=0.3, alpha=0.6) # Horizontal lines
            ax.plot(grid_z_np[i::grid_res_vis, 0], grid_z_np[i::grid_res_vis, 1], color='gray', linewidth=0.3, alpha=0.6) # Vertical lines
        # Plot transformed data points
        ax.scatter(z_np[:, 0], z_np[:, 1], s=5, alpha=0.4, c='blue', label='f(X)')

    ax.set_title('Latent Space Z (f(X)) & Warped Grid', fontsize=10)
    ax.set_xlabel('z1', fontsize=9); ax.set_ylabel('z2', fontsize=9)
    ax.grid(True, alpha=0.3); ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(-z_lim, z_lim); ax.set_ylim(-z_lim, z_lim)
    ax.tick_params(axis='both', which='major', labelsize=8)

    # --- Bottom Right: Latent Samples (Base Distribution) ---
    ax = axes[1, 1]
    z_samples = torch.randn(num_samples, model.dim).to(device)
    z_samples_np = z_samples.cpu().numpy()
    ax.scatter(z_samples_np[:, 0], z_samples_np[:, 1], s=5, alpha=0.4, c='red', label='Z ~ N(0,I)')
    ax.set_title('Latent Space Z (Samples)', fontsize=10)
    ax.set_xlabel('z1', fontsize=9); ax.set_ylabel('z2', fontsize=9)
    ax.grid(True, alpha=0.3); ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(-z_lim, z_lim); ax.set_ylim(-z_lim, z_lim)
    ax.tick_params(axis='both', which='major', labelsize=8)

    # --- Bottom Left: Generated Data & Warped Grid (Latent -> Data) ---
    ax = axes[1, 0]
    with torch.no_grad():
        x_generated = model.inverse(z_samples)
        x_generated_np = x_generated.cpu().numpy()
        z_range = np.linspace(-z_lim, z_lim, grid_res_vis)
        grid_zx, grid_zy = np.meshgrid(z_range, z_range)
        grid_latent = torch.from_numpy(np.stack([grid_zx.ravel(), grid_zy.ravel()], axis=1).astype(np.float32)).to(device)
        grid_x_inv = model.inverse(grid_latent)
        grid_x_inv_np = grid_x_inv.cpu().numpy()
        # Plot warped grid lines
        for i in range(grid_res_vis):
            ax.plot(grid_x_inv_np[i*grid_res_vis:(i+1)*grid_res_vis, 0], grid_x_inv_np[i*grid_res_vis:(i+1)*grid_res_vis, 1], color='gray', linewidth=0.3, alpha=0.6) # Horizontal
            ax.plot(grid_x_inv_np[i::grid_res_vis, 0], grid_x_inv_np[i::grid_res_vis, 1], color='gray', linewidth=0.3, alpha=0.6) # Vertical
        # Plot generated data points
        ax.scatter(x_generated_np[:, 0], x_generated_np[:, 1], s=5, alpha=0.4, c='red', label='X = f$^{-1}$(Z)')

    ax.set_title('Generated Data Space (X = f$^{-1}$(Z))', fontsize=10)
    ax.set_xlabel('x1 (scaled)', fontsize=9); ax.set_ylabel('x2 (scaled)', fontsize=9)
    ax.grid(True, alpha=0.3); ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(x_lim); ax.set_ylim(y_lim) # Use same limits as input data plot
    ax.tick_params(axis='both', which='major', labelsize=8)

    # Save the frame
    frame_path = os.path.join(frame_dir, f"{dataset_name}_epoch_{epoch:04d}.png")
    print(f"  Saving frame {epoch}: Saving to {frame_path}...")
    plt.savefig(frame_path, dpi=100) # Adjust dpi if needed
    plt.close(fig) 
    model.train() # Set model back to training mode

Train the Model

In [15]:
def train_model_with_animation(model, dataloader, optimizer, num_epochs, loss_fn,
                               frame_save_interval=10, dataset_name="Dataset", device='cpu', frame_dir="anim_frames",
                               grid_res_vis=20): 
    print(f"--- Training on {dataset_name} for {num_epochs} epochs ---")
    os.makedirs(frame_dir, exist_ok=True)
    model.train()
    viz_data = dataloader.dataset.data.to(device)

    for epoch in range(1, num_epochs + 1):
        epoch_loss = 0.0
        disable_inner_tqdm = (epoch % frame_save_interval == 0 or epoch == num_epochs)
        pbar = tqdm(dataloader, desc=f"Epoch {epoch}/{num_epochs}", leave=False, disable=disable_inner_tqdm)

        for batch_idx, x in enumerate(pbar):
            x = x.to(device)
            optimizer.zero_grad()
            z, log_det = model(x)
            loss = loss_fn(z, log_det)
            # --- NAN CHECK ---
            if torch.isnan(loss).any() or torch.isinf(loss).any():
                print(f"\nNaN/Inf loss detected at Epoch {epoch}, Batch {batch_idx}! Stopping.")
                return # Exit training
            if torch.isnan(z).any() or torch.isinf(z).any():
                 print(f"\nNaN/Inf in Z detected at Epoch {epoch}, Batch {batch_idx}! Stopping.")
                 return

            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            if not disable_inner_tqdm: 
                pbar.set_postfix({"Loss": f"{loss.item():.4f}"})

        avg_epoch_loss = epoch_loss / len(dataloader)

        # Save animation frame periodically
        if epoch % frame_save_interval == 0 or epoch == num_epochs:
             # Pass grid_res_vis to the saving function
             save_animation_frame(model, viz_data, dataset_name, epoch, device, frame_dir, grid_res_vis=grid_res_vis)
             print(f"Epoch {epoch:<5} Avg Loss: {avg_epoch_loss:.4f} | Frame Saved")
        elif epoch % 100 == 0: # Print loss less frequently otherwise
             print(f"Epoch {epoch:<5} Avg Loss: {avg_epoch_loss:.4f}")

    print(f"--- Finished Training on {dataset_name} ---")

In [16]:
# --- Main Execution ---
if __name__ == "__main__":
    # Hyperparameters
    N_SAMPLES = 2000
    BATCH_SIZE = 512
    LEARNING_RATE = 1e-3
    NUM_EPOCHS = 300 # Increase  epochs for potentially better convergence
    FRAME_SAVE_INTERVAL = 5 # Save frame every 5 epochs -> 60 frames total
    NUM_COUPLING_LAYERS = 8 
    HIDDEN_UNITS = 128     
    DEPTH = 3             # MLP depth in coupling layers
    VIDEO_FPS = 15        # Frames per second for the output video
    GRID_RES_VIS = 20     # Lower resolution for visualization grid

    DATASETS_TO_RUN = ['moons', 'circles']
    ANIMATION_DIR = "anim_frames_2d_videos" 
    all_frame_files = {} # Dictionary to store frame filenames for each dataset

    for name in DATASETS_TO_RUN:
        print(f"\n{'='*10} Processing Dataset: {name.capitalize()} {'='*10}")
        dataset_frame_dir = os.path.join(ANIMATION_DIR, name)

        # Load data
        data_np, scaler = load_and_preprocess_data(name, N_SAMPLES)
        dataset = Dataset2D(data_np)
        dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

        # Create model and optimizer
        model = RealNVP_2D(dim=2,
                           num_coupling_layers=NUM_COUPLING_LAYERS,
                           hidden_units=HIDDEN_UNITS,
                           depth=DEPTH).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

        # Train and save frames, passing the visualization grid resolution
        train_model_with_animation(model, dataloader, optimizer, NUM_EPOCHS, nll_loss,
                                   frame_save_interval=FRAME_SAVE_INTERVAL,
                                   dataset_name=name.capitalize(),
                                   device=device,
                                   frame_dir=dataset_frame_dir,
                                   grid_res_vis=GRID_RES_VIS) # Pass the lower resolution

        all_frame_files[name] = sorted([os.path.join(dataset_frame_dir, f) for f in os.listdir(dataset_frame_dir) if f.endswith(".png")])

    print("\n--- All Training Complete ---")
    print("\n--- Creating MP4 videos from saved frames ---")
    for name, frame_files in all_frame_files.items():
        if frame_files: # Check if list is not empty
            video_path = os.path.join(ANIMATION_DIR, f"transformation_{name}.mp4") # Changed extension
            print(f"Creating MP4 video for {name.capitalize()} at {video_path} ({len(frame_files)} frames)...")
            try:
                # Use imageio.mimwrite to create the video
                # Load frames first to ensure they exist and are readable
                frames_data = [imageio.v3.imread(filename) for filename in tqdm(frame_files, desc=f"Loading {name} frames")]
                imageio.mimwrite(video_path, frames_data, fps=VIDEO_FPS, codec='libx264', quality=8, macro_block_size=1)
                print(f"MP4 video saved.")
            except Exception as e:
                print(f"\n-------------------------------------")
                print(f"Error creating MP4 video for {name}: {e}")
                print("Ensure imageio and ffmpeg backend are installed correctly.")
                print("Try: pip install imageio-ffmpeg")
                print("-------------------------------------")
        else:
            print(f"No frames found for {name.capitalize()}, skipping video creation.")


========== Processing Dataset: Moons ==========
--- Training on Moons for 300 epochs ---


Epoch 1/300:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 2/300:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 3/300:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 4/300:   0%|          | 0/4 [00:00<?, ?it/s]

...

  Saving frame 300: Saving to anim_frames_2d_videos/circles/Circles_epoch_0300.png...
Epoch 300   Avg Loss: 2.0042 | Frame Saved
--- Finished Training on Circles ---

--- All Training Complete ---

--- Creating MP4 videos from saved frames ---
Creating MP4 video for Moons at anim_frames_2d_videos/transformation_moons.mp4 (60 frames)...


Loading moons frames:   0%|          | 0/60 [00:00<?, ?it/s]

MP4 video saved.
Creating MP4 video for Circles at anim_frames_2d_videos/transformation_circles.mp4 (60 frames)...


Loading circles frames:   0%|          | 0/60 [00:00<?, ?it/s]

MP4 video saved.


Completed 